# 1.1 Importando bibliotecas

In [1]:
import pandas as pd 
import numpy as np

# 1.2 Lendo a planilha

In [16]:
df = pd.read_csv("../../data/raw/DH_2022_Br_Regioes_20240423.xlsx - Faixa De Renda.csv")

# 1.3 Visualizar primeiras linhas

In [10]:
df.head()

,N,Regiao,Faixa de Renda,Domicílios Rústicos,Domicílios Improvisados,Habitação Precária,Cômodos,Unidades conviventes déficit,Coabitação,Ônus,Déficit Habitacional (Habitação Precária + Coabitação + Ônus)
0,1,Norte,Faixa 1,86.349,209.547,295.896,11.012,86.551,97.562,167.438,560.896
1,1,Norte,Faixa 2,19.607,4.174,23.781,5.247,66.914,72.160,17.227,113.168
2,1,Norte,Faixa 3,8.714,439,9.153,418.000,58.288,58.706,-,67.860
3,1,Norte,Acima da Faixa 3,2.323,109,2.432,245.000,28.727,28.973,-,31.405
4,2,Nordeste,Faixa 1,238.086,422.657,660.743,11.017,137.939,148.955,657.418,1.467.116


# 1.4.1 Renomear colunas

In [17]:
df.rename(columns={
    'Regiao' : 'regiao',
    'Faixa de Renda' : 'faixa_de_renda',
    'Domicílios Rústicos' : 'domicilios_rusticos',
    'Domicílios Improvisados' : 'domicilios_improvisados',
    'Habitação Precária' : 'habitacao_precaria',
    'Cômodos' : 'comodos',
    'Unidades conviventes déficit' : 'unidades_conviventes_deficit',
    'Coabitação' : 'coabitacao',
    'Ônus' : 'onus',
    'Déficit Habitacional (Habitação Precária + Coabitação + Ônus)' : 'deficit_original'
}, inplace=True)

df.head()

,N,regiao,faixa_de_renda,domicilios_rusticos,domicilios_improvisados,habitacao_precaria,comodos,unidades_conviventes_deficit,coabitacao,onus,deficit_original
0,1,Norte,Faixa 1,86.349,209.547,295.896,11.012,86.551,97.562,167.438,560.896
1,1,Norte,Faixa 2,19.607,4.174,23.781,5.247,66.914,72.160,17.227,113.168
2,1,Norte,Faixa 3,8.714,439,9.153,418.000,58.288,58.706,-,67.860
3,1,Norte,Acima da Faixa 3,2.323,109,2.432,245.000,28.727,28.973,-,31.405
4,2,Nordeste,Faixa 1,238.086,422.657,660.743,11.017,137.939,148.955,657.418,1.467.116


# 1.4.2 Removendo pontos e convertendo para inteiros

In [18]:
colunas_excluir = ["regiao", "faixa_de_renda"]

colunas_converter = [col for col in df.columns if col not in colunas_excluir]

for col in colunas_converter:
    df[col] = (
        df[col]
        .astype(str).
        str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
    )
    df[col] = pd.to_numeric(df[col], errors='coerce')

print(df.dtypes)
df.head()

N                                 int64
regiao                           object
faixa_de_renda                   object
domicilios_rusticos               int64
domicilios_improvisados           int64
habitacao_precaria                int64
comodos                           int64
unidades_conviventes_deficit      int64
coabitacao                        int64
onus                            float64
deficit_original                  int64
dtype: object


,N,regiao,faixa_de_renda,domicilios_rusticos,domicilios_improvisados,habitacao_precaria,comodos,unidades_conviventes_deficit,coabitacao,onus,deficit_original
0,1,Norte,Faixa 1,86349,209547,295896,11012,86551,97562,167438.0,560896
1,1,Norte,Faixa 2,19607,4174,23781,5247,66914,7216,17227.0,113168
2,1,Norte,Faixa 3,8714,439,9153,4180,58288,58706,NaN,67860
3,1,Norte,Acima da Faixa 3,2323,109,2432,2450,28727,28973,NaN,31405
4,2,Nordeste,Faixa 1,238086,422657,660743,11017,137939,148955,657418.0,1467116


# 1.5 Tratamento de valores ausentes

In [19]:
df['onus'] = df['onus'].fillna(0)
df.head()

,N,regiao,faixa_de_renda,domicilios_rusticos,domicilios_improvisados,habitacao_precaria,comodos,unidades_conviventes_deficit,coabitacao,onus,deficit_original
0,1,Norte,Faixa 1,86349,209547,295896,11012,86551,97562,167438.0,560896
1,1,Norte,Faixa 2,19607,4174,23781,5247,66914,7216,17227.0,113168
2,1,Norte,Faixa 3,8714,439,9153,4180,58288,58706,0.0,67860
3,1,Norte,Acima da Faixa 3,2323,109,2432,2450,28727,28973,0.0,31405
4,2,Nordeste,Faixa 1,238086,422657,660743,11017,137939,148955,657418.0,1467116


# 1.6 Validação e correção da coluna de déficit total

In [20]:
df['deficit_total'] = df['habitacao_precaria'] + df['coabitacao'] + df['onus']
df.drop(columns=['N', 'deficit_original'], inplace=True)

df.head()

,regiao,faixa_de_renda,domicilios_rusticos,domicilios_improvisados,habitacao_precaria,comodos,unidades_conviventes_deficit,coabitacao,onus,deficit_total
0,Norte,Faixa 1,86349,209547,295896,11012,86551,97562,167438.0,560896.0
1,Norte,Faixa 2,19607,4174,23781,5247,66914,7216,17227.0,48224.0
2,Norte,Faixa 3,8714,439,9153,4180,58288,58706,0.0,67859.0
3,Norte,Acima da Faixa 3,2323,109,2432,2450,28727,28973,0.0,31405.0
4,Nordeste,Faixa 1,238086,422657,660743,11017,137939,148955,657418.0,1467116.0


# 1.7 Separação dos dados: Regiões vs. Brasil

In [21]:
df_regioes = df[df['regiao'] != 'Brasil'].copy()
df_brasil = df[df['regiao'] == 'Brasil'].copy()

print(df_regioes.head())
print(df_brasil.head())

     regiao    faixa_de_renda  domicilios_rusticos  domicilios_improvisados  \
0     Norte           Faixa 1                86349                   209547   
1     Norte           Faixa 2                19607                     4174   
2     Norte           Faixa 3                 8714                      439   
3     Norte  Acima da Faixa 3                 2323                      109   
4  Nordeste           Faixa 1               238086                   422657   

   habitacao_precaria  comodos  unidades_conviventes_deficit  coabitacao  \
0              295896    11012                         86551       97562   
1               23781     5247                         66914        7216   
2                9153     4180                         58288       58706   
3                2432     2450                         28727       28973   
4              660743    11017                        137939      148955   

       onus  deficit_total  
0  167438.0       560896.0  
1   17227.

# 1.8 Verificando os dados após limpeza: 

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   regiao                        24 non-null     object 
 1   faixa_de_renda                24 non-null     object 
 2   domicilios_rusticos           24 non-null     int64  
 3   domicilios_improvisados       24 non-null     int64  
 4   habitacao_precaria            24 non-null     int64  
 5   comodos                       24 non-null     int64  
 6   unidades_conviventes_deficit  24 non-null     int64  
 7   coabitacao                    24 non-null     int64  
 8   onus                          24 non-null     float64
 9   deficit_total                 24 non-null     float64
dtypes: float64(2), int64(6), object(2)
memory usage: 2.0+ KB
